In [10]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

#incarcare baza de date CIFAR
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#Afisare forma /dimensiunea bazei da date
print('x_train shape', x_train.shape)
print(x_train.shape[0], 'train samples')
print (x_test.shape[0], 'test samples')

#hot encode
num_classes =10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape (50000, 32, 32, 3)
50000 train samples
10000 test samples


# Acum sa cream straturile pentru a reproduce AlexNet

In [11]:
l2_reg =0

#initializeaza modelul
model = Sequential()

#primul strat Conv
model.add(Conv2D(96, (11,11), input_shape=x_train.shape[1:], 
           padding='same', kernel_regularizer=l2(l2_reg)   
             ))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

#al doilea strat Conv
model.add(Conv2D(256, (5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

#al 3-lea strat Conv
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

#al 4-lea strat Conv
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(1024, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

#al 5-lea strat conv
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(1024,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#prilul strat FC
model.add(Flatten())
model.add(Dense(3072))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

#al 2-lea strat FC
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

#al 3-lea strat FC
model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

print(model.summary())

#compli

model.compile(keras.optimizers.Adadelta(), 'categorical_crossentropy', ['accuracy'])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 32, 32, 96)        34944     
_________________________________________________________________
batch_normalization_25 (Batc (None, 32, 32, 96)        384       
_________________________________________________________________
activation_25 (Activation)   (None, 32, 32, 96)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 16, 16, 256)       614656    
_________________________________________________________________
batch_normalization_26 (Batc (None, 16, 16, 256)       1024      
_________________________________________________________________
activation_26 (Activation)   (None, 16, 16, 256)      

# Instruirea AlexNet in baza de date CIFAR10

In [12]:
#Parametri de instruire
batch_size = 32
epochs = 1

#instruirea modelui
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test,y_test),
                    shuffle=True
)

#Salvare model instruit
model.save("c:/Users/Harum/Documents/13/CIFAR10_AlexNet_1_Epoch.h5")

#Evaluarea performantei modelului de instruire
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])


ResourceExhaustedError: OOM when allocating tensor with shape[16384,3072] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node dense_5/random_uniform/RandomUniform (defined at C:\Users\Harum\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:4357) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_5/random_uniform/RandomUniform', defined at:
  File "C:\Users\Harum\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Harum\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Harum\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Harum\AppData\Roaming

# R